In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='xxxx', project_access_token='xxxx')
pc = project.project_context

(備考)　上のセルは自動生成されたもの

# PyTorchのモデルをWatson MLにデプロイする

## アップロード用モデルの作り方

Google Colabで下記リンク先のNotebookを実行   

https://github.com/makaishi2/sample-data/blob/master/notebooks/cifar10_pytorch.ipynb

その後、次のコードをセルに追加して実行

```py3
# ダミーデータの作成
dummy_input = torch.randn((1, 3, 32, 32)).to(device)

# onyx形式でexport
# keep_initializers_as_inputsのオプションが重要でこれがないとエラーになる
torch.onnx.export(net, dummy_input, "cifar10-pytorch-sample.onnx", 
                  keep_initializers_as_inputs=True, verbose=True)

# gz形式に圧縮
!tar czvf cifar10-pytorch-sample.gz cifar10-pytorch-sample.onnx

# 結果をPCにダウンロード
fn = 'cifar10-pytorch-sample.gz'
from google.colab import files
files.download(fn) 
```


In [ ]:
# project-libを使ってデータアセットのファイルをローカルにコピーする

fn = 'cifar10-pytorch-sample.gz'
infile = project.get_file(fn)
with open(fn, 'wb') as local_file:
    local_file.write(infile.read())

In [ ]:
!ls -l

# 1. モデルの保存

## 1.1 必要ライブラリの導入

In [ ]:
# 必要ライブラリの導入
!pip install -U ibm-watson-machine-learning | tail -n 1

## 1.2 APIClient インスタンスの生成とデプロイメントスペースの関連付

In [ ]:
#  ロケーションの指定
# ロケーションは下記のコマンドの結果得られたものを使う
#
# $ ibmcloud resource service-instance

location = 'us-south'  # Dallas
#location = 'jp-tok'    # Tokyo
#location = 'au-syd'  # Sydney

# APIKeyの指定
# API key は次のリンク先から生成する
#
# https://cloud.ibm.com/iam/apikeys

apikey = "xxxx"

wml_credentials = {
    "apikey": apikey,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [ ]:
# API Clientインスタンスの生成

from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [ ]:
# space_idの取得
client.spaces.list()

In [ ]:
# 上の結果を基に手で設定する
space_id = '9336c80e-574d-442c-a031-90765b925770'

In [ ]:
# デプロイメントスペースIDの設定
client.set.default_space(space_id)

## 1.3 Software Specification ID の取得

In [ ]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.7")

## 1.4 モデルの保存

In [ ]:
model_path = fn

In [ ]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'External pytorch model',
            client.repository.ModelMetaNames.TYPE: 'pytorch-onnx_1.3',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

In [ ]:
published_model = client.repository.store_model(
    model=model_path,
    meta_props=metadata)

## 1.5 モデルの詳細確認

In [ ]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:
models_details = client.repository.list_models()

# 2. モデルのデプロイ

## 2.1 モデルのデプロイ

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of external pytorch model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

In [ ]:
# デプロイメントの一覧表示

client.deployments.list()

In [ ]:
# depolyment_uid 取得

deployment_uid = client.deployments.get_uid(created_deployment)

# すでにデプロイ済みの場合、下記コメントをはずして手動でdeployment_uidを設定する

# deployment_uid = "e3e5d8e9-e768-42f4-9c94-beb9beb61150"

# deployment_uidの確認
print(deployment_uid)

## 2.2 デプロイメントの詳細確認

In [ ]:
client.deployments.get_details(deployment_uid)

# 3. 予測 (Watson MLライブラリを利用)

## 3.1 検証用データの取得

In [ ]:
!pip install torchvision==0.8.1 | tail -n 1

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
# 分類先クラス名
classes = ['plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# 分類クラス数
num_classes = len(classes)

# 1回の学習で何枚の画像を使うか
batch_size = 128

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

## 3.2 検証用データの組み立てと、イメージ表示

In [ ]:
N = 100

values = []
labels = []

plt.figure(figsize=(15, 15))

for i in range(N):
    # テスト用データの組み立て
    image, label = testset[i]
    labels.append(label)
    xdata = image.numpy().tolist()
    values .append(xdata)
    
    # データの画面表示
    ax = plt.subplot(10, 10, i + 1)
    img = np.transpose(image.numpy(), (1, 2, 0))
    img2 = (img + 1)/2 
    plt.imshow(img2)
    ax.set_title(classes[label], fontsize=10)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

In [ ]:
scoring_payload = {"input_data": [{"values": values}]}

## 3.3 予測APIの呼び出し

In [ ]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

In [ ]:
w1 = predictions['predictions'][0]['values']
w2 = np.array([np.argmax(x) for x in w1])
print('予測: ',w2)
print('正解: ', labels)

In [ ]:
# 混同行列の計算

# 必要ライブラリの取込み
from sklearn.metrics import confusion_matrix

# 混同行列の生成
#   y_test: 検証データの正解データ
#   y_pred: 検証データの予測結果
matrix = confusion_matrix(labels, w2)

In [ ]:
# 混同行列表示用関数
import pandas

def make_cm(matrix, columns):
    # matrix numpy配列
    
    # columns 項目名リスト
    n = len(columns)
    
    # '正解データ'をn回繰り返すリスト生成
    act = ['正解データ'] * n
    pred = ['予測結果'] * n
    
    #データフレーム生成
    cm = pd.DataFrame(matrix, 
        columns=[pred, columns], index=[act, columns])
    return cm

In [ ]:
from IPython.display import display

# make_cmを使った混同行列標示
cm = make_cm(matrix, classes)
display(cm)